In [43]:
import requests

instructions = ['کمترین قیمت ست سطل و جای دستمال لالیپاپ مدل رنگین کمان چنده؟',
                "لطفاً دراور چهار کشو (کد D14) را برای من تهیه کنید.",
                "عرض پارچه تریکو جودون 1/30 لاکرا گردباف نوریس به رنگ زرد طلایی چقدر است؟",
                "کمترین قیمت در این پایه برای گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ چقدر است؟",
                "کدام یک از این ماگ‌های خرید ماگ-لیوان هندوانه فانتزی و کارتونی کد 1375 یا ماگ لته خوری سرامیکی با زیره کد 741 دارای سبک کارتونی و فانتزی بوده و برای کودکان یا نوجوانان مناسب‌تر است؟",
                "قیمت متوسط خردکن سیلور کرست مدل NF-1923 تیغه تیتانیومی در شهر قم چقدر است؟",
                """محصول \'لحاف کرسی دست دوز ترمه طرح نسترن رنگ زرشکی\' با شناسه "iushix" و \'لحاف کرسی دست دوز ترمه آبی طرح نسترن کد 119807 Lahaf\' از نظر در دسترس بودن گارانتی در شهرهای مختلف را مقایسه کنید. کدام یک گزینه بهتری است؟""",
                "من به پایه دیواری مدلی برایت F3010 که مناسب تلویزیون\u200cهای سایز ۱۷ تا ۵۵ اینچ باشد، نیاز دارم.'",
                "سلام. لطفاً پاف نیمکت انتظار سه نفره با ستون\u200cبندی اسفنج مبلی و رنگ\u200cبندی مختلف را که امکان ارسال به سراسر ایران دارد، برای من تهیه کنید. متشکرم.",
                'میز جلو مبلی اسمردیس مدل J131 از چه موادی ساخته شده است؟',
                'لطفاً درباره ظرفیت ماگ سرامیکی دسته رنگی طرح نشانگر سوخت – Coffee Fuel Gauge – اطلاعات بیشتری بدهید و بفرمایید که این ماگ چه ظرفیتی دارد؟',
                'سلام! من دنبال یه گیاه بونسای هستم که خیلی خاص و زیبا باشه. می\u200cخوام برای هدیه دادن استفاده کنم و بهتره که ارسال گل رایگان هم داشته باشه. قیمتش هم حدوداً بین ۳,۷۰۰,۰۰۰ تا ۴,۱۰۰,۰۰۰ تومان باشه. می\u200cتونید کمکم کنید؟',
                'در مقایسه بین "چرخ خیاطی ژانومه دیجیتال (۱۲مدل دوخت) جا دکمه اتومات مدلSchone" و "چرخ خیاطی ژانومه مدل Juki 9800 اتوماتیک" کدامیک از نظر قیمت متوسط در مشهد بهتر است؟',
]
# chat_id = "retrieve_similar"
chat_id = "mytest"

url = "https://behnam-torob-ai-shopping-assistant.darkube.app/chat"
data = {
    "chat_id": chat_id,
    "messages": [{"type": "text", "content":  instructions[6] }]
}

# Disable system proxy
resp = requests.post(url, json=data, proxies={"http": None, "https": None})

In [44]:
print(resp)

<Response [200]>


In [45]:
resp.json()

{'message': "محصول 'لحاف کرسی دست دوز ترمه ابی طرح نسترن کد 119807' گزینه بهتری از نظر در دسترس بودن گارانتی در شهرهای مختلف است.",
 'base_random_keys': ['iushix', 'yfmqhv'],
 'member_random_keys': None}

In [46]:
print(resp.json()['message'])

محصول 'لحاف کرسی دست دوز ترمه ابی طرح نسترن کد 119807' گزینه بهتری از نظر در دسترس بودن گارانتی در شهرهای مختلف است.
